In [ ]:
from tensorflow import keras
import keras.backend as K
import tensorflow as tf
import tensorflow.keras
import numpy as np
import pickle
import os
from keras import optimizers
from pathlib import Path
from collections import Counter
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

from utils import *
from models.GRU import GRU_model
from models.GRU_D import create_grud_model, load_grud_model
from models.APC import *

In [ ]:
#load dataset
with open('datasets/physionet2012.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [ ]:
#get class_balance
classes_total_dataset = np.concatenate([data["train"]["y_classes"], data["val"]["y_classes"], data["test"]["y_classes"]])
class_balance = Counter(classes_total_dataset)
class_balance = {c: class_balance[c] / classes_total_dataset.shape[0] for c in class_balance}
# get class weights
class_weights = get_class_weights(class_balance)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                  patience=5, min_lr=0.001)

## GRU 

In [ ]:
def GRU_experiment(data, class_balance, config, save_model_path, model_name, method = "class_weights", class_weights=0):
    
    K.clear_session()
    Path(save_model_path).mkdir(parents=True, exist_ok=True)
    
    # create model
    model = GRU_model(x_length=config["x_length"], 
                      n_features=config["n_features"], 
                      n_aux=config["n_aux"], 
                      n_classes=config["n_classes"], 
                      n_neurons=config["n_neurons"], 
                      learning_rate=config["learning_rate"], 
                      dropout_rate=config["dropout_rate"], 
                      recurrent_dropout=config["recurrent_dropout"], 
                      loss_type=config["loss_type"])
    
    checkpoint = ModelCheckpoint(save_model_path, monitor=config["loss_to_monitor"], verbose=0, save_best_only=True, mode=config["monitor_mode"])
    
    #train model
    #undersample majority class
    if method == "undersample":
        #make training set balanced
        X_train, X_train_aux, y_train = make_classes_balanced(data["train"])
        history = model.fit([X_train, X_train_aux], 
                            y_train, 
                            validation_data=([data["val"]["X_val"], data["val"]["X_aux"]], data["val"]["y"]), 
                            epochs=config["epochs"], 
                            callbacks=[reduce_lr, checkpoint], 
                            batch_size=config["batch_size"], 
                            verbose=2)
    #class weights
    elif method == "class_weights":
        history = model.fit([data["train"]["X_train"], data["train"]["X_aux"]], data["train"]["y"], 
                            validation_data=([data["val"]["X_val"], data["val"]["X_aux"]], data["val"]["y"]), 
                            epochs=config["epochs"], 
                            callbacks=[reduce_lr, checkpoint], 
                            batch_size=config["batch_size"], 
                            class_weight=class_weights, 
                            verbose=2)
        
    #use oversampling batch generator
    elif method == "oversample":
        print("using batch generator")
        #training steps
        steps_per_epoch = np.ceil(data["train"]["X_train"].shape[0] / batch_size)
        # validation steps
        validation_steps= np.ceil(data["val"]["X_val"].shape[0] /batch_size)
        
        training_generator = oversample_batch_generator_GRU(data["train"], class_balance, config["epochs"], config["batch_size"])
        validation_generator = batch_generator_GRU(data["val"], config["batch_size"])
        history = model.fit_generator(generator=training_generator, 
                                      validation_data=validation_generator,
                                      epochs = config["epochs"],
                                      steps_per_epoch = steps_per_epoch,
                                      validation_steps = validation_steps,
                                      callbacks=[reduce_lr, checkpoint],  
                                      verbose=2)
  
    #load best weights
    model.load_weights(save_model_path)
    
    # get results
    yhat_test = model.predict([data["text"]["X_test"], data["test"]["X_aux"]], verbose=1)
    y_pred_test = [np.argmax(y, axis=None, out=None) for y in yhat_test]
    y_test_ = [np.argmax(y, axis=None, out=None) for y in y_test]

    results = get_results_df(y_test_, y_pred_test, y_test, yhat_test, model_name, config["n_classes"])
    
    return results

In [ ]:
config = {"x_length": 48,
          "n_neurons": 64,
          "learning_rate": 1e-3, 
          "dropout_rate": 0.0,
          "recurrent_dropout": 0.0,
          "n_features": 74,
          "n_aux": 9,
          "n_classes": 2,
          "loss_type": "binary_crossentropy",
          "loss_to_monitor": "val_auprc",
          "monitor_mode": "max",
          "epochs": 50, 
          "batch_size": 32
}

- ### undersample majority class

In [ ]:
## add save best model path
GRU_undersamp_model_path = "saved_models/GRU_undersampling"

In [ ]:
GRU_experiment(data, class_balance, config, GRU_undersamp_model_path, "GRU_undersampling", method="undersample", class_weights=0)

- ### class weights

In [ ]:
## add save best model path
GRU_cw_model_path = "saved_models/GRU_cw"

In [ ]:
GRU_experiment(data, class_balance, config, GRU_cw_model_path, "GRU_cw", method="class_weights", class_weights=class_weights)

- ### oversample minority class

In [ ]:
## add save best model path
GRU_over_model_path = "saved_models/GRU_oversampling"

In [ ]:
GRU_experiment(data, class_balance, config, GRU_over_model_path, "GRU_oversampling", method="oversample", class_weights=0)

## GRU - D

In [ ]:
config = {"x_length": 48,
          "n_neurons": 32,
          "learning_rate": 1e-3, 
          "dropout_rate": 0.1,
          "recurrent_dropout": 0.0,
          "n_features": 37,
          "n_aux": 9,
          "n_classes": 2,
          "loss_type": "binary_crossentropy",
          "loss_to_monitor": "val_auprc",
          "monitor_mode": "max",
          "epochs": 50, 
          "batch_size": 32,
          "eval_metric": auprc
}

In [ ]:
def GRU_ext_experiment(data, config, save_model_path, class_weights, predefined_model = "GRUD"):
    K.clear_session()
    Path(save_model_path).mkdir(parents=True, exist_ok=True)
    
    #create model
    GRU_ext = create_grud_model(input_dim=config["n_features"],
                                aux_dim=config["n_aux"],
                              hidden_neurons=config["n_neurons"],
                              dropout_rate=config["dropout_rate"], 
                              recurrent_dropout_rate=config["recurrent_dropout_rate"],
                              output_dim=config["n_classes"],
                              predefined_model=predefined_model)

    #compile model
    adam_optim = optimizers.Adam(lr=config["learning_rate"]) 
    model.compile(loss=config["loss_type"], optimizer=adam_optim, metrics=[config["eval_metric"]])
    
    #save best weights
    checkpoint = ModelCheckpoint(save_model_path, monitor=config["loss_to_monitor"], verbose=0, save_best_only=True, save_weights_only=True, mode=config["monitor_mode"])
    
    print("using batch generator")
    #training steps
    train_steps_per_epoch = np.ceil(data["train"]["X"].shape[0] / config["batch_size"])
    # validation steps
    validation_steps= np.ceil(data["val"]["X"].shape[0] / config["batch_size"])
    #train
    training_generator = batch_generator_GRUD(data["train"], config["batch_size"])
    validation_generator = batch_generator_GRUD(data["val"], config["batch_size"])
    history = model.fit_generator(generator=training_generator, 
                                  validation_data=validation_generator,
                                  epochs = config["epochs"],
                                  steps_per_epoch = steps_per_epoch,
                                  validation_steps = validation_steps,
                                  callbacks=[reduce_lr, checkpoint],  
                                  class_weight=class_weights, 
                                  verbose=2)
    #load best weights
    model.load_weights(save_model_path)
    
    test_steps_per_epoch = np.ceil(data["test"]["X"].shape[0] / config["batch_size"])
    
    y_test_pred = model.predict_generator(batch_generator_GRUD(data["test"], config["batch_size"]), steps=test_steps_per_epoch)
    
    y_pred_test_ = [np.argmax(y, axis=None, out=None) for y in y_test_pred]
    y_test_ = [np.argmax(y, axis=None, out=None) for y in data["test"]["y"]]

    results = get_results_df(y_test_, y_pred_test_, y_test, y_test_pred, predefined_model, config["n_classes"])
    
    return results

In [ ]:
save_model_path = "saved_models/GRUD_model"

In [ ]:
GRU_ext_experiment(data, config, save_model_path, class_weights, predefined_model = "GRUD")

## GRU - Mean

In [ ]:
save_model_path = "saved_models/GRUD_mean_model"

In [ ]:
GRU_ext_experiment(data, config, save_model_path, class_weights, predefined_model = "GRUmean")

## GRU - Forward 

In [ ]:
save_model_path = "saved_models/GRUD_forward_model"

In [ ]:
GRU_ext_experiment(data, config, save_model_path, class_weights, predefined_model = "GRUforward")

## GRU - Simple

In [ ]:
save_model_path = "saved_models/GRUD_simple_model"

In [ ]:
GRU_ext_experiment(data, config, save_model_path, class_weights, predefined_model = "GRUsimple")

## GRU - APC

In [ ]:
def APC_experiment(data, encoder, config, class_weights, save_weights_path, step=1, stop_APC_grad=False):

    K.clear_session()  
    Path(save_weights_path).mkdir(parents=True, exist_ok=True)
      
    model = create_APC_classifier(config, encoder, stop_APC_grad)
    if config["pre_trained_weights"] != 0:
        model.load_weights(config["pre_trained_weights"])
    
    
    #train model
    #training steps
    train_steps_per_epoch = np.ceil(data["train"]["X"].shape[0] / config["batch_size"])
    # validation steps
    validation_steps= np.ceil(data["val"]["X"].shape[0]/ config["batch_size"])
    
    # add checkpoint to save best model weights
    checkpoint = ModelCheckpoint(save_weights_path, save_weights_only=True, monitor=config["loss_to_monitor"], verbose=0, save_best_only=True, mode=config["monitor_mode"])
    
    if encoder == "GRU":
        training_generator = APC_batch_generator(data["train"], config["time_shift"], config["batch_size"])
        validation_generator = APC_batch_generator(data["val"], config["time_shift"], config["batch_size"])
    elif encoder == "GRUD":
        training_generator = APC_GRUD_batch_generator(data["train"], config["time_shift"], config["batch_size"])
        validation_generator = APC_GRUD_batch_generator(data["val"], config["time_shift"], config["batch_size"])

    history = model.fit_generator(generator=training_generator, 
                                  validation_data=validation_generator,
                                  epochs = config["epochs"],
                                  steps_per_epoch = train_steps_per_epoch,
                                  validation_steps = validation_steps,
                                  callbacks=[reduce_lr, checkpoint], 
                                  class_weight={ 'output_1': {0: 1 , 1: 1} , 'output_2': class_weights},  
                                  verbose=2)
    if step == 1:
        #pretraining step
        return print("done training.")
    
    else: # step 2 or 3
        # load best weights
        model.load_weights(save_weights_path)
        
        test_steps_per_epoch = np.ceil(data["text"]["X"].shape[0] / config["batch_size"])
        
        if encoder == "GRU":
            _, y_pred = model.predict_generator(APC_batch_generator(data["test"], config["time_shift"], config["batch_size"]), steps=test_steps_per_epoch)
        elif encoder == "GRUD":
            test_generator = APC_GRUD_batch_generator(data["test"], config["time_shift"], config["batch_size"])
            class_pred, actual_classes = [], []
            i = 0
            for batch in test_generator:
                if i < (steps_per_epoch + 1):
                    actual_classes.append(batch[1])
                    r, c = model.predict_on_batch(batch[0])
                    class_pred.append(c)
                    i += 1
                else:
                    break
                    
            y_test = [x[1] for i, x in enumerate(actual_classes)]
            y_test = np.vstack(y_test)
            y_pred = np.vstack(class_pred)
            
        y_pred_classes = [np.argmax(y, axis=None, out=None) for y in y_pred]
        y_test_classes = [np.argmax(y, axis=None, out=None) for y in y_test]
        model_name = encoder + "- APC"
        results = get_results_df(y_test_classes, y_pred_classes, y_test, y_pred, model_name, config["n_classes"])
        return results

In [ ]:
config = {"n_features": 74,
                  "n_neurons": 64,
                  "learning_rate": 1e-03,
                  "aux_dim": 9,
                  "n_classes": 2,
                  "dropout_rate": 0.0,
                  "recurrent_dropout_rate": 0.0,
                  "batch_size": 32,
                  "epochs": 100,
                  "l1": 1,
                  "l2": 0,
                  "l1_type": masked_mse
                  "l2_type": 'binary_crossentropy',
                  "evaluation_metric": auprc}

In [ ]:
# add/change time_shift_factor n
# we run our experiments for n = 0 (autoencoder), 1, 2, 3, 4 & 5
time_shift_factor = 1
config["time_shift"] = time_shift_factor

### Step 1: Pre-train APC

In [ ]:
## add save best model path
APC_step1_weights_path = "apc_models/APC_step1_n{}_weights_{}".format(time_shift_factor, encoder_model)

In [ ]:
config["l1"] = 1
config["l2"] = 0
config["epochs"] = 100
config["pre_trained_weights"] = 0

config["loss_to_monitor"] = "val_loss"
config["monitor_mode"] = "min"

In [ ]:
APC_experiment(data, "GRU", config, class_weights, 0, step=1, stop_APC_grad=False)

### Step 2: Frozen: Train classifier with frozen APC weights

In [ ]:
config["l1"] = 0
config["l2"] = 1
config["learning_rate"] = 1e-4
config["pre_trained_weights"] = APC_step1_weights_path

config["loss_to_monitor"] = "val_dense_3_auprc"
config["monitor_mode"] = "max"

In [ ]:
APC_step2_weights_path = "apc_models/APC_step2_n{}_weights_{}".format(time_shift_factor, encoder_model)

In [ ]:
APC_step2_results = APC_experiment(data, "GRU", config, class_weights, APC_step2_weights_path, step=2, stop_APC_grad=True)

### Step 3: Fine-tuned: Train APC + classifier end-to-end

In [ ]:
config["l1"] = 1
config["l2"] = 1
config["pre_trained_weights"] = APC_step2_weights_path

In [ ]:
APC_step3_weights_path = "apc_models/APC_step3_n{}_weights_{}".format(time_shift_factor, encoder_model)

In [ ]:
APC_step3_results = APC_experiment(data, "GRU", config, class_weights, APC_step3_weights_path, step=3, stop_APC_grad=False)